Importing the Dependencies

In [26]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thakk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
#Printing The English Stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [29]:
#Load Data into a pd df
news_df = pd.read_csv("Data/train.csv")

In [30]:
news_df.shape

(20800, 5)

In [31]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [32]:
#Total missing values in each column
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [33]:
#Fill the missing values with null string
news_df = news_df.fillna('')

In [34]:
#Merge author and title column in content column
news_df['content'] = news_df['author'] + ' ' + news_df['title']
news_df['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [35]:
#Splitting the data
X = news_df.drop(columns = 'label', axis = 1)
Y = news_df['label']

In [36]:
port_stem = PorterStemmer()

In [37]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [38]:
news_df['content'] = news_df['content'].apply(stemming)

In [40]:
X = news_df['content'].values
Y = news_df['label'].values

In [41]:
#convert the text to numbers
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [63]:
#Train, Test, Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [64]:
#Training the Model
model = LogisticRegression()

In [65]:
model.fit(X_train, Y_train)

LogisticRegression()

In [84]:
#Accuracy Score on train data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [85]:
print('Accuracy score: ', training_data_accuracy)

Accuracy score:  0.9870793269230769


In [86]:
#Accuracy Score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [87]:
print('Accuracy score: ', test_data_accuracy)

Accuracy score:  0.9788461538461538


In [88]:
import pickle

In [89]:
with open('model_pickle', 'wb') as f:
    pickle.dump(model, f)

In [90]:
with open('model_pickle', 'rb') as f:
    mp = pickle.load(f)

In [96]:
print(X_train)

  (0, 16996)	0.08248905284665282
  (0, 15582)	0.1026625072097838
  (0, 15295)	0.08093765979242373
  (0, 13694)	0.36950429289685166
  (0, 12137)	0.23011372594955526
  (0, 11531)	0.3326884338802441
  (0, 10351)	0.23464937631443447
  (0, 10306)	0.07973556517365195
  (0, 9025)	0.22393729206339982
  (0, 6431)	0.22472203370772692
  (0, 5510)	0.3243126421709088
  (0, 4250)	0.15524384511402725
  (0, 3918)	0.2722774713215564
  (0, 3070)	0.29829505674623263
  (0, 3048)	0.2914515366228229
  (0, 144)	0.36950429289685166
  (1, 17001)	0.2982286004271752
  (1, 11604)	0.24443193760155407
  (1, 11259)	0.2401116344011407
  (1, 10578)	0.25701993358841674
  (1, 5400)	0.23516611260508916
  (1, 2943)	0.20671312276804724
  (1, 2652)	0.34560004325665117
  (1, 2459)	0.32999315220275083
  (1, 1346)	0.31632469087599513
  :	:
  (16637, 9427)	0.2420374399790419
  (16637, 5767)	0.4365717896085534
  (16637, 4250)	0.19346056362964573
  (16637, 2123)	0.4604659767019617
  (16638, 16996)	0.09735854886431025
  (16638, 16